In [ ]:
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Data Generator

Data Generator

Data Generator 

```
# In generators
```





```
`# This is formatted as code`
```

# Data Generator

In [ ]:
class DataGenerator(object):
    
    def __init__(self,code_path,commnet_path,test_persentage,max_code_len, max_commnet_len):
        self.raw_code = []
        self.raw_comment = []

        self.code_data = []
        self.comment_data = []
        
        self.train_code_data = []
        self.train_comment_data = []
        
        self.test_code_data = []
        self.test_comment_data = []
        self.test_raw_comment = []
        self.gCodes = []
        self.gComment = []
        self.index2word = dict()
        
        self.ReadData(code_path, commnet_path,max_code_len, max_commnet_len)
        self.SplitData(test_persentage)
        
        
        self.vocab_size = len(self.index2word)
    def getTestData(self):
        return self.test_code_data,self.test_comment_data, self.test_raw_comment
    def getGdata(self):
        return self.gCodes,self.gComment
    def ReadData(self, code_path, commnet_path, max_code_len, max_commnet_len):
        code_lines=[]
        comment_lines=[]
        raw_comment_lines = []
        with open(code_path) as fin:
            code_lines = fin.readlines()
        with open(commnet_path) as fin:
            comment_lines = fin.readlines()
        with open('/content/drive/My Drive/CodeSum/dataset/comment_preproc_F.txt') as fin:
            raw_comment_lines = fin.readlines()  #reading pre written comments

        gCodes = []
        gComment = []
        with open('/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt') as f: #
            for line in f:
                gCodes.append([str(c) for c in line.rstrip().split(',')])
        with open('/content/drive/My Drive/CodeSum/dataset/comment_f_indexed.txt') as f:
            for line in f:
                gComment.append([str(c) for c in line.rstrip().split(',')])

        for code_line, comment_line, raw_comment_line,code,comment in zip(code_lines,comment_lines,raw_comment_lines,gCodes,gComment):
            tokens = code_line.split(",")
            words = comment_line.split(",")
            if len(tokens) > max_code_len or len(words) > max_commnet_len:
                continue
            if len(tokens) <= 12 or len(words) <= 3:
                continue
            tokens = [int(token) for token in tokens]
            self.code_data.append(tokens)
            words = [int(token) for token in words]
            self.comment_data.append(words)

            self.raw_comment.append(raw_comment_line)
            self.gCodes.append(code)
            self.gComment.append(comment)
        np.random.seed(30)
        np.random.shuffle(self.code_data)
        
        np.random.seed(30)
        np.random.shuffle(self.comment_data)

        np.random.seed(30)
        np.random.shuffle(self.raw_comment)
        np.random.seed(30)
        np.random.shuffle(self.gCodes)
        np.random.seed(30)
        np.random.shuffle(self.gComment)
        print ("num of data:",len(self.code_data))
        with open("/content/drive/My Drive/CodeSum/dataset/comment_f_keyword_Vocab.txt") as fin: #removing stopwords from pre written comments
            for i, word in enumerate(fin):
                self.index2word[i] = word.rstrip() #assigning index to words
    def SplitData(self,test_persentage):
        
        self.train_code_data = self.code_data[:int(len(self.code_data) * (1 - test_persentage))]
        self.train_comment_data = self.comment_data[:int(len(self.comment_data) * (1 - test_persentage))]
        
        self.test_code_data = self.code_data[int(len(self.code_data) * (1 - test_persentage)):]
        self.test_comment_data = self.comment_data[int(len(self.comment_data) * (1 - test_persentage)):]
        self.test_raw_comment = self.raw_comment[int(len(self.raw_comment) * (1 - test_persentage)):]

        self.gCodes = self.gCodes[int(len(self.gCodes) * (1 - test_persentage)):]
        self.gComment = self.gComment[int(len(self.gComment) * (1 - test_persentage)):]

    def MakeDataset(self, train):

        codes = []
        keywords = []

        if train:
            codes = self.train_code_data
            keywords = self.train_comment_data

        else:
            codes = self.test_code_data
            keywords = self.test_comment_data

        train_codes = []
        train_keywords = []

        next_words = np.zeros((len(keywords), self.vocab_size), dtype=np.bool)

        for i, (code, keyword) in enumerate(zip(codes, keywords)):
            train_codes.append(code)
            keyword = set(keyword)
            for index in keyword:
                next_words[i,index] = 1

        return train_codes, next_words
                

Training model

In [ ]:
import tensorflow as tf

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#import tensorflow as tf
import tensorflow.compat.v1 as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True, log_device_placement=True))

#from groundhog.trainer import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D
from keras.layers.core import Activation, Flatten, Dense, Dropout, RepeatVector
from keras.layers.pooling import MaxPooling1D
from keras.optimizers import RMSprop
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
#from keras.engine.topology import Merge
from keras.preprocessing import sequence
import time
from keras import callbacks
from skimage.feature.tests.test_util import plt
import keras

#import data_generator


max_caption_len = 20
maxlen = 600

data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_keyword_indexed.txt", 0.20,
                                        maxlen, max_caption_len)

codes, next_words = data_gen.MakeDataset(train=True)
#
codes = sequence.pad_sequences(codes, maxlen=maxlen)


codesT, next_wordsT = data_gen.MakeDataset(train=False)
codesT = sequence.pad_sequences(codesT, maxlen=maxlen)


vocab_size = 4000

max_features = 6000  # size of token

nb_output = data_gen.vocab_size  # len(Y_train[0])

nb_filter = 128
filter_length = 10
hidden_dims = 256
embedding_dims = 128

code_model = Sequential()
code_model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
code_model.add(Dropout(0.2))
#def Convolution1D(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
code_model.add(Convolution1D(filters=nb_filter,kernel_size=10,padding='valid',dilation_rate=1))
code_model.add(Activation('relu'))

code_model.add(Convolution1D(filters=nb_filter,kernel_size=5,padding='valid',dilation_rate=1))
code_model.add(Activation('relu'))

code_model.add(Convolution1D(filters=nb_filter, kernel_size=3,padding='valid',dilation_rate=1))
code_model.add(Activation('relu'))
#MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last', **kwargs)

code_model.add(MaxPooling1D(pool_size=code_model.output_shape[1]))
code_model.add(Flatten())
# We add a vanilla hidden layer:
code_model.add(Dense(256))
code_model.add(Activation('relu'))

code_model.add(Dropout(0.5))
code_model.add(Dense(nb_output))
code_model.add(Activation('sigmoid'))

code_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])


print ("train")#
early = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
cp = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/CodeSum/dataset/keyword_f.h5', monitor='val_loss', verbose=0, save_best_only=True,
                                     save_weights_only=True, mode='auto')

del data_gen

code_model.fit(codes, next_words, batch_size=32, epochs=500, validation_data=(codesT, next_wordsT), callbacks=[early])

json_string = code_model.to_json()
open('/content/drive/My Drive/CodeSum/dataset/keyword_f.json', 'w').write(json_string)
code_model.save_weights('/content/drive/My Drive/CodeSum/dataset/keyword_f.h5')

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

num of data: 25072
train
Epoch 1/500
627/627 [==============================] - 509s 812ms/step - loss: 0.0219 - accuracy: 0.1791 - val_loss: 0.0098 - val_accuracy: 0.1870
Epoch 2/500
627/627 [==============================] - 512s 817ms/step - loss: 0.0101 - accuracy: 0.1810 - val_loss: 0.0097 - val_accuracy: 0.1844
Epoch 3/500
627/627 [==============================] - 514s 820ms/step - loss: 0.0099 - accuracy: 0.2609 - val_loss: 0.0094 - val_accuracy: 0.3428
Epoch 4/500
627/627 [==============================] - 514s 820ms/step - loss: 0.0095 - accuracy: 0.3801 - val_loss: 0.0092 - val_accuracy: 0.4098
Epoch 5/500
627/627 [==============================] - 515s 821ms/step - loss: 0.0092 - accuracy: 0.4030 - val_loss: 0.0088 - val_accuracy: 0.4225
Epoch 6/500
627/627 [==============================] - 512s 817ms/step - loss: 0.0089 - accuracy: 0.4163 - val_loss: 0.0086 - val_accuracy: 0.4375
E

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#import tensorflow as tf
import tensorflow.compat.v1 as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
#gpu_options=tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True, log_device_placement=True))
#from groundhog.trainer import SGD
#from tensorflow import keras
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.convolutional import Convolution1D
from tensorflow.python.keras.layers.core import Activation, Flatten, Dense, Dropout, RepeatVector
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.python.keras.layers.wrappers import TimeDistributed
#from tensorflow.python.keras.layers.core import Merge
from tensorflow.python.keras.preprocessing import sequence
#import keras
import time
from tensorflow.python.keras import callbacks
from skimage.feature.tests.test_util import plt




Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



In [ ]:
!pip install groundhog.trainer

ERROR: Could not find a version that satisfies the requirement groundhog.trainer (from versions: none)
ERROR: No matching distribution found for groundhog.trainer


In [ ]:
!python setup.py develop

python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
max_caption_len = 20
maxlen = 600

In [ ]:
data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_keyword_indexed.txt", 0.20, maxlen, max_caption_len)
#giving files with words from code and comment converted to index also same word/ token has same index
codes, next_words = data_gen.MakeDataset(train=True)
#
codes = sequence.pad_sequences(codes, maxlen=maxlen)


codesT, next_wordsT = data_gen.MakeDataset(train=False)
codesT = sequence.pad_sequences(codesT, maxlen=maxlen)




num of data: 25072


In [ ]:
vocab_size = 4000

max_features = 6000  # size of token

nb_output = data_gen.vocab_size  # len(Y_train[0])

nb_filter = 128
filter_length = 10
hidden_dims = 256
embedding_dims = 128

In [ ]:
!pip install keras==1.0.1

     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for keras: filename=Keras-1.0.1-cp36-none-any.whl size=132078 sha256=b647bbd70c61b7e667fc4838379db7681d11e9370b9acdce8598da8feec956e8
  Stored in directory: /root/.cache/pip/wheels/e9/4d/8e/3964c381b769bb3286d75c98e0e0c9c78fd63821e7018c0b36
Successfully built keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 1.0.1 which is incompatible.
ERROR: kapre 0.1.3.1 has requirement keras>=2.0.0, but you'll have keras 1.0.1 which is incompatible.
ERROR: fancyimpute 0.4.3 has requirement keras>=2.0.0, but you'll have keras 1.0.1 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
code_model = Sequential()
code_model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
code_model.add(Dropout(0.2))
#def Convolution1D(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
code_model.add(Convolution1D(filters=nb_filter,kernel_size=10,padding='valid',dilation_rate=1))
code_model.add(Activation('relu'))

code_model.add(Convolution1D(filters=nb_filter,kernel_size=5,padding='valid',dilation_rate=1))
code_model.add(Activation('relu'))

code_model.add(Convolution1D(filters=nb_filter, kernel_size=3,padding='valid',dilation_rate=1))
code_model.add(Activation('relu'))
#MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last', **kwargs)

code_model.add(MaxPooling1D(pool_size=code_model.output_shape[1]))
code_model.add(Flatten())
# We add a vanilla hidden layer:
code_model.add(Dense(256))
code_model.add(Activation('relu'))

code_model.add(Dropout(0.5))
code_model.add(Dense(nb_output))
code_model.add(Activation('sigmoid'))

code_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])


In [ ]:
import tensorflow as tf

In [ ]:
print ("train")
early = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
cp = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/CodeSum/dataset/keyword_f.h5', monitor='val_loss', verbose=0, save_best_only=True,
                                     save_weights_only=True, mode='auto')

train


In [ ]:
del data_gen

code_model.fit(codes, next_words, batch_size=32, epochs=500, validation_data=(codesT, next_wordsT), callbacks=[early])

json_string = code_model.to_json()
open('/content/drive/My Drive/CodeSum/dataset/keyword_f.json', 'w').write(json_string)
code_model.save_weights('/content/drive/My Drive/CodeSum/dataset/keyword_f.h5')

NameError: ignored

Keyword Predictor 

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import tensorflow.compat.v1 as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True, log_device_placement=True))


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



In [ ]:
from tensorflow.python.keras.models import model_from_json, Sequential
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
import math
import logging
import sys
import time
import nltk
#import data_generator
from scipy.stats.mstats_basic import sen_seasonal_slopes



In [ ]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)


In [ ]:
class SentenceGeneration(object):
    def __init__(self):
        self.model = Sequential()
        self.index2word = dict()
        self.word2Index = dict()
        self.index2token = dict()
        self.token2Index = dict()
        with open("/content/drive/My Drive/CodeSum/dataset/comment_f_keyword_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                word = word.rstrip()             #saving indexes to words and vise-versa
                self.index2word[i] = word
                self.word2Index[word] = int(i)
        with open("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                word = word.rstrip()            #saving indexes to tokens and
                self.index2token[i] = word
                self.token2Index[word] = int(i)

    # def __init__(self,codes,targets):
    #     self.model = Sequential()
    #     # self.codes = codes
    #     # self.targets = targets

    def readModel(self, name):
        model = model_from_json(open('/content/drive/My Drive/CodeSum/dataset/'+name + '.json').read())
        model.load_weights('/content/drive/My Drive/CodeSum/dataset/'+name + '.h5')
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

        self.model = model

    def printSentence(self, indices):
        return ' '.join([self.index2word[x] for x in indices])

        #for index in indices:
        #    print self.index2word[index],
    def returnCode(self, indices):

        return ' '.join([self.index2token[x-1] for x in indices])

    def removeToken(self,indices):
        indices = indices[1:-1]
        unk_index = self.word2Index['UNK']
        if unk_index in indices:
            indices.remove(unk_index)
        return indices

    def generateSentence(self, code, n):

        code = sequence.pad_sequences([code], maxlen=600)


        pred = self.model.predict(code)[0]
        sorted_pred = np.argsort(pred)[::-1]

        pred_word = [ self.index2token[index] for index in sorted_pred[:n]]
        return pred_word

In [ ]:
def rAtk(pred, target, k):
    #sorted_pred = np.argsort(pred)[::-1]
    correct = 0
    for i in pred[:k]:
        if i in target:
            correct += 1

    return correct / float(len(target))


In [ ]:
gen = SentenceGeneration()


data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_keyword_indexed.txt",0.20, 600, 20)
gen.readModel('keyword_f')

num of data: 25072


In [ ]:
codes,keywords,raw_comment= data_gen.getTestData()


In [ ]:
np.random.seed(30)
np.random.shuffle(codes)
np.random.seed(30)
np.random.shuffle(raw_comment)
np.random.seed(30)
np.random.shuffle(keywords)


In [ ]:
sens = []
co = []
comm = []
r = 0
k = 30

In [ ]:
for i,(code,comment) in enumerate(zip(codes,keywords)):
    #keyword = gen.generateSentence(code,5)
    c = sequence.pad_sequences([code], maxlen=600)
    pred = gen.model.predict(c)[0]
    #pred[52] = 0

    sorted_pred = np.argsort(pred)[::-1]

    sens.append([gen.index2word[s] for s in sorted_pred[:k]])
    co.append(gen.returnCode(code))
    comm.append(gen.printSentence(comment))

    r += rAtk(sorted_pred, comment, k)



    sys.stdout.write('\r' + str(i)+' score :'+str(r/(i+1)))

5014 score :0.650133887800147

In [ ]:
with open('/content/drive/My Drive/CodeSum/dataset/keyword_prediction4.txt', 'w') as fin:
    fin.write(str(r/(i+1)))
    fin.write("\n")
    for i in range(len(co[:300])):
        s = str(co[i])
        code = s.replace("{","{\n").replace(";",";\n").replace("}","}\n")
        fin.write("code:\n"+ code)
        fin.write("comment:\n"+ raw_comment[i].rstrip()+ '\n')
        fin.write("--generate--\n")

        fin.write(' '.join(sens[i])+'\n')
        fin.write("\n")
    # for sen in sens:
    #     fin.write(gen.printSentence(sen[1:-1])+'\n')

MODEL

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.convolutional import Convolution1D
from tensorflow.python.keras.layers.core import Activation, Flatten, Dense, Dropout, RepeatVector
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.python.keras.optimizers import RMSprop, Adam
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.python.keras.layers.wrappers import TimeDistributed
#from tensorflow.python.keras.engine.topology import Merge



In [ ]:
import logging
import sys
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.engine.training import Model

In [ ]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [ ]:
class CGModel(object):
    '''
    classdocs
    '''

    #def __init__(self, max_features=20000, code_maxlen=600, embed_size=100, hidden_size=200, vocab_size=4000 , optimiser, weights=None, gru=False, clipnorm=-1, batch_size=None,t=None, lr=0.001):
    def __init__(self, max_features, code_maxlen, embed_size, hidden_size, vocab_size , optimiser, weights, gru, clipnorm, batch_size,t, lr):
        
        self.max_t = t  # Expected timesteps. Needed to build the Theano graph

        # Model hyperparameters
        self.max_features = 20000
        self.vocab_size = vocab_size  # size of word vocabulary
        self.embed_size = 100  # number of units in a word embedding
        self.hidden_size = 200 # number of units in first LSTM

        # Optimiser hyperparameters
        self.optimiser = optimiser  # optimisation method
        self.lr = lr
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.clipnorm = clipnorm

        self.weights = weights  # initialise with checkpointed weights?
        
        
        


    def buildModel(self):
        '''
        Define the exact structure of your model here. We create an image
        description generation model by merging the VGG image features with
        a word embedding model, with an LSTM over the sequences.
        '''
        logger.info('Building Keras model...')

                
        
        code_model = Sequential()
        code_model.add(Embedding(self.max_features, self.embed_size, input_length=maxlen))#,dropout=0.2))
     #    Convolution1D(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, groups=1, 
    #activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', 
    #kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
        code_model.add(Convolution1D(filters=nb_filter,kernel_size=filter_length, padding='valid',  groups=1))
        #model.add(BatchNormalization())
        code_model.add(Activation('relu'))
        
        code_model.add(Convolution1D(filters=nb_filter,kernel_size=filter_length,padding='valid',groups=1))
        # model.add(BatchNormalization())
        code_model.add(Activation('relu'))
        
        code_model.add(MaxPooling1D(pool_size=code_model.output_shape[1]))
        code_model.add(Flatten())
        # We add a vanilla hidden layer:
        code_model.add(Dense(128))
        
        
        # next, let's define a RNN model that encodes sequences of words
        # into sequences of 128-dimensional word vectors.
        language_model = Sequential()
        language_model.add(Embedding(vocab_size, 256, input_length=max_caption_len))
        language_model.add(LSTM(128, return_sequences=True))
        language_model.add(TimeDistributed(Dense(128)))
        
        # let's repeat the image vector to turn it into a sequence.
        code_model.add(RepeatVector(max_caption_len))
        
        # the output of both models will be tensors of shape (samples, max_caption_len, 128).
        # let's concatenate these 2 vector sequences.
        model = Sequential()
        #x = tf.keras.layers.Concatenate(axis=-1) (inputs=[t,h,mem_encoder])
        model.add(tf.keras.layers.Concatenate(axis =-1)(inputs=[code_model , language_model ]) )
        # let's encode this vector sequence into a single vector
        model.add(LSTM(256, return_sequences=False))
        # which will be used to compute a probability
        # distribution over what the next word in the caption should be!
        model.add(Dense(self.vocab_size))
        model.add(Activation('softmax'))
        
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])
        
        
        #model.fit([codes, partial_captions], next_words, batch_size=16, nb_epoch=100)
        

**dropout check pls, groups or dilation rate**

Generating Data

In [ ]:
import numpy as np
from tensorflow.python.keras.models import model_from_json, Sequential
from tensorflow.python.keras.preprocessing import sequence

In [ ]:
class SentenceGeneration(object):
    def __init__(self):
        self.model = Sequential()
        self.index2word = dict()
        self.word2Index = dict()
        self.index2token = dict()
        self.token2Index = dict()
        with open("/content/drive/My Drive/CodeSum/dataset/comment_f_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                word = word.rstrip()
                self.index2word[i] = word
                self.word2Index[word] = int(i)
        with open("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                word = word.rstrip()
                self.index2token[i] = word
                self.token2Index[word] = int(i)

    # def __init__(self,codes,targets):
    #     self.model = Sequential()
    #     # self.codes = codes
    #     # self.targets = targets

    def readModel(self, name):
        model = model_from_json(open('/content/drive/My Drive/CodeSum/dataset/'+name + '.json').read())
        model.load_weights('/content/drive/My Drive/CodeSum/dataset/'+name + '.h5')
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

        self.model = model

    def printSentence(self, indices):
        return ' '.join([self.index2word[x] for x in indices])

        #for index in indices:
        #    print self.index2word[index],
    def returnCode(self, indices):

        return ' '.join([self.index2token[x-1] for x in indices])

    def removeToken(self,indices):
        indices = indices[1:-1]
        unk_index = self.word2Index['UNK']
        if unk_index in indices:
            indices.remove(unk_index)
        return indices





    def generateSentence(self, code, n):

        code = sequence.pad_sequences([code], maxlen=600)


        pred = self.model.predict(code)[0]
        sorted_pred = np.argsort(pred)[::-1]

        pred_word = [ self.index2token[index] for index in sorted_pred[:n]]
        return pred_word

In [ ]:
class DataGenerator(object):
    def __init__(self, code_path, commnet_path, test_persentage, max_code_len, max_commnet_len):
        self.raw_code = []
        self.raw_comment = []

        self.code_split_plag = []

        self.code_data = []
        self.comment_data = []

        self.train_code_data = []
        self.train_comment_data = []

        self.valid_code_data = []
        self.valid_comment_data = []

        self.test_code_data = []
        self.test_comment_data = []
        self.test_raw_comment = []

        self.index2word = dict()

        self.ReadData(code_path, commnet_path, max_code_len, max_commnet_len)
        self.SplitData(test_persentage)

        self.vocab_size = len(self.index2word)

    def getTestData(self):
        return self.test_code_data, self.test_comment_data, self.test_raw_comment

    def getTestID(self):
        with open('/content/drive/My Drive/CodeSum/codenn/data/stackoverflow/csharp/dev/codenn.txt') as f:
            ids = [val.split('\t')[0] for val in f]
        return ids
  #data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_indexed.txt", 0.20,maxlen, max_caption_len)
    def ReadData(self, code_path, commnet_path, max_code_len, max_commnet_len):
        code_lines = []
        comment_lines = []
        raw_comment_lines = []
        code_split_plags = []
        with open(code_path) as fin:
            code_lines = fin.readlines()
        with open(commnet_path) as fin:
            comment_lines = fin.readlines()
        with open('/content/drive/My Drive/CodeSum/dataset/comment_preproc_F.txt') as fin:
            raw_comment_lines = fin.readlines()

        with open('/content/drive/My Drive/CodeSum/dataset/code_f_keyword_Vocab.txt') as fin: #getting eerrors
            code_split_plags = fin.readlines()
            #qnaSet
#error line error line
        count = 0
        for code_line, comment_line, raw_comment_line, plag in zip(code_lines, comment_lines, raw_comment_lines,
                                                                   code_split_plags):
            tokens = code_line.split(",")
            words = comment_line.split(",")
            # if len(tokens) > max_code_len or len(words) > max_commnet_len:
            #    continue

            if (len(tokens) <= 12 or len(words) <= 3) and plag.rstrip() != 'test':
                continue

            tokens = [int(token) for token in tokens]
            self.code_data.append(tokens)
            words = [int(token) for token in words]
            self.comment_data.append(words)

            self.raw_comment.append(raw_comment_line)
            self.code_split_plag.append(str(plag).rstrip())

        # np.random.seed(30)
        # np.random.shuffle(self.code_data)

        # np.random.seed(30)
        # np.random.shuffle(self.comment_data)

        # np.random.seed(30)
        # np.random.shuffle(self.raw_comment)
        print ("num of data:", len(self.code_data))

        with open("/content/drive/My Drive/CodeSum/dataset/comment_f_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                self.index2word[i] = word.rstrip()

    def SplitData(self, test_persentage):

        for code, comment, rawComment, flag in zip(self.code_data, self.comment_data, self.raw_comment,
                                                   self.code_split_plag):
            if flag == "train":
                self.train_code_data.append(code)
                self.train_comment_data.append(comment)
            elif flag == "valid":
                self.valid_code_data.append(code)
                self.valid_comment_data.append(comment)
            else:
                self.test_code_data.append(code)
                self.test_comment_data.append(comment)
                self.test_raw_comment.append(rawComment)


    def MakeDataset(self,train,divide=1,part=0):

        codes = []
        captions = []

        if train == True:
            c = len(self.train_code_data)/divide
            if divide == part+1:
                codes = self.train_code_data[c*part:]
                captions = self.train_comment_data[c*part:]
            else:
                codes = self.train_code_data[c * part:c * (part+1)]
                captions = self.train_comment_data[c * part:c * (part+1)]
        else:
            codes = self.test_code_data
            captions = self.test_comment_data

        train_codes = []
        patial_captions = []
        next_words = []


        for code, caption in zip(codes,captions):
            for i in range(1,len(caption)):
                next_word = [0 for j in range(self.vocab_size)]
                patial_caption = caption[:i]
                next_word[int(caption[i])] = 1

                train_codes.append(code)
                patial_captions.append(patial_caption)
                next_words.append(next_word)

        return train_codes, patial_captions, next_words

    def MakeDataset3(self, train, divide=1, part=0):

        codes = []
        captions = []

        if train:
            codes = self.train_code_data
            captions = self.train_comment_data

        else:
            codes = self.test_code_data
            captions = self.test_comment_data

        train_codes = []
        patial_captions = []
        next_words = []

        for code, caption in zip(codes, captions):
            for i in range(1, len(caption)):
                patial_caption = caption[:i]
                next_words.append(int(caption[i]))
                train_codes.append(code)
                patial_captions.append(patial_caption)



        next_words2 = np.zeros((len(next_words), self.vocab_size), dtype=np.bool)
        for i,index in enumerate(next_words):
            next_words2[i,index] = 1

        return train_codes, patial_captions, next_words2

    def MakeDataset4(self, train, mem_size):

        codes = []
        captions = []

        if train:
            codes = self.train_code_data
            captions = self.train_comment_data

        else:
            codes = self.test_code_data
            captions = self.test_comment_data

        train_codes = []
        patial_captions = []
        next_words = []

        gen = SentenceGeneration()
       #error coming here #gen.readModel('qnaData_all_keyword')
        gen.readModel('keyword_f')
        predict_keywords = []

        for code, caption in zip(codes, captions):
            c = sequence.pad_sequences([code], maxlen=500)
            pred = gen.model.predict(c)[0]

            sorted_pred = np.argsort(pred)[::-1]

            for i in range(1, len(caption)):
                patial_caption = caption[:i]
                next_words.append(int(caption[i]))

                train_codes.append(code)
                patial_captions.append(patial_caption)
                predict_keywords.append([s for s in sorted_pred[:mem_size]])

        next_words2 = np.zeros((len(next_words), self.vocab_size), dtype=np.bool)
        for i,index in enumerate(next_words):
            next_words2[i, index] = 1
        return train_codes, patial_captions, next_words2, predict_keywords

    def MakeDataset2(self, train):
        codes = []
        captions = []

        if train:

            codes = self.train_code_data
            captions = self.train_comment_data
        else:
            codes = self.test_code_data
            captions = self.test_comment_data

        train_codes = []
        patial_captions = []


        for j,(code, caption) in enumerate(zip(codes, captions)):
            for i in range(1, len(caption)):


                patial_caption = caption[:i]

                train_codes.append(code)
                patial_captions.append(patial_caption)

        next_words = np.zeros((len(train_codes), self.vocab_size), dtype=np.bool)

        for j,(code, caption) in enumerate(zip(codes, captions)):
            for i in range(1, len(caption)):

                next_words[j, int(caption[i])] = 1

        return train_codes, patial_captions, next_words

     
                
                
             



'''
X_train = X_data[:int(len(X_data) * (1 - 0.25))]
Y_train = Y_data[:int(len(X_data) * (1 - 0.25))]
X_test = X_data[int(len(X_data) * (1 - 0.25)):]
Y_test = Y_data[int(len(X_data) * (1 - 0.25)):]
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
'''

'\nX_train = X_data[:int(len(X_data) * (1 - 0.25))]\nY_train = Y_data[:int(len(X_data) * (1 - 0.25))]\nX_test = X_data[int(len(X_data) * (1 - 0.25)):]\nY_test = Y_data[int(len(X_data) * (1 - 0.25)):]\nX_train = sequence.pad_sequences(X_train, maxlen=maxlen)\nX_test = sequence.pad_sequences(X_test, maxlen=maxlen)\n'

**#gen.readModel('qnaData_all_keyword')**

**qnaSet dataset not found**

Training Attmem2

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow.compat.v1 as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True, log_device_placement=True))

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.convolutional import Convolution1D
from tensorflow.python.keras.layers.core import Activation, Flatten, Dense, Dropout, RepeatVector
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense

#import data_generator
from tensorflow.python.keras.preprocessing import sequence
import keras

from tensorflow.python.keras.models import Model

#from bleu import computeMaps, bleuFromMaps
#from text_generator import SentenceGeneration

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



In [ ]:
max_caption_len = 26
maxlen = 500
mem_size = 30


In [ ]:
data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_indexed.txt", 0.20,maxlen, max_caption_len)

codes, partial_captions, next_words, predict_words= data_gen.MakeDataset4(train=True, mem_size=mem_size)


num of data: 4885


In [ ]:
codes = sequence.pad_sequences(codes, maxlen=maxlen)
partial_captions = sequence.pad_sequences(partial_captions, maxlen=max_caption_len)
predict_words = sequence.pad_sequences(predict_words, maxlen=mem_size)

codesT, partial_captionsT, next_wordsT, predict_wordsT = data_gen.MakeDataset4(train=False, mem_size=mem_size)
codesT = sequence.pad_sequences(codesT, maxlen=maxlen)
partial_captionsT = sequence.pad_sequences(partial_captionsT, maxlen=max_caption_len)
predict_wordsT = sequence.pad_sequences(predict_wordsT, maxlen=mem_size)

In [ ]:
vocab_size = 5000

max_features = 10000  # size of token

nb_output = data_gen.vocab_size  # len(Y_train[0])

nb_filter = 128
filter_length = 5
hidden_dims = 256
embedding_dims = 128

mem_size = 30

NameError: ignored

In [ ]:
import tensorflow as tf
import keras

In [ ]:
embedding_for_nse = tf.compat.v1.keras.layers.Embedding(vocab_size,128,Dropout(0.3),input_length=max_caption_len)
#embedding_for_nse(Dropout(0.3))
#def tf.compat.v1.keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', 
#embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, 
#input_length=None, **kwargs)

language_input = tf.compat.v1.keras.Input(shape=(max_caption_len,), dtype= 'float32')
print(language_input)
language_embedding = embedding_for_nse(language_input)
vocab_input = Input(shape=(mem_size,), dtype= 'float32')
vocab_embedding = embedding_for_nse(vocab_input)


match = tf.keras.layers.Dot(axes=[2,2])([language_embedding,vocab_embedding])#,dot_axes=[2,2])
match = Activation('softmax')(match)

mem_encoder = tf.keras.layers.Dot(axes=[2,1])(inputs=[match,vocab_embedding])#,dot_axes=[2,1])
mem_encoder = Activation('relu')(mem_encoder) # mem : memory embedding



Tensor("input_11:0", shape=(None, 20), dtype=float32)


TypeError: ignored

In [ ]:
# the output of both models will be tensors of shape (samples, max_caption_len, 128).
# let's encode this vector sequence into a single vector
h = LSTM(400, return_sequences=True)(language_embedding)
# which will be used to compute a probability
# distribution over what the next word in the caption should be!

In [ ]:
code_input = Input(shape=(maxlen,),dtype='float32')
print(code_input)
embedding_for_code = Embedding(max_features, 400, maxlen,Dropout(0.5))(code_input)
#embedding_for_code=(Dropout(0.5))
code_match = tf.keras.layers.Dot(axes=[2,2]) (inputs=[h,embedding_for_code])#,dot_axes=[2,2])
code_match = Activation('softmax')(code_match)
code_attention = tf.keras.layers.Dot(axes=[2,1])(inputs=[code_match,embedding_for_code])#,dot_axes=[2,1])
code_attention = Activation('relu')(code_attention) # mem : memory embedding




Tensor("input_10:0", shape=(None, 500), dtype=float32)


ValueError: ignored

**Dot(axes) error**

In [ ]:
#code_model.add(Convolution1D(filters=nb_filter,kernel_size=10,padding='valid',dilation_rate=1))
#code_model.add(Activation('relu'))

code_model = Convolution1D(filters=nb_filter,kernel_size=10,padding='valid',dilation_rate=1)(code_attention)
code_model = Activation('relu')(code_attention)

code_model = Convolution1D(filters=nb_filter,kernel_size=5,padding='valid',dilation_rate=1)(code_model)
code_model = Activation('relu')(code_model)

code_model = Convolution1D( filters=nb_filter,kernel_size=3,padding='valid',dilation_rate=1)(code_model)
code_model = Activation('relu')(code_model)
#def MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last', **kwargs)
code_model = MaxPooling1D (pool_size=code_model.shape[1])(code_model)
code_model = Flatten()(code_model)
# We add a vanilla hidden layer:
code_model = Dense(400)(code_model)


t = Dense(400)(code_attention)

h = Dense(400)(h)

In [ ]:
import tensorflow as tf

In [ ]:
x = tf.keras.layers.Concatenate(axis=-1) (inputs=[t,h,mem_encoder])
x = LSTM(400, return_sequences=False)(x)

x = Dense(400)(x)

x = Dropout(0.5)(x)
predictions = Dense(nb_output,activation='softmax')(x)
optimizer = RMSprop(lr=0.0001)
model = tf.keras.Model ([code_input,language_input,vocab_input], predictions)

In [ ]:
model.compile (loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])


In [ ]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding_37 (Embedding)        (None, 26, 128)      640000      input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 600)]        0                                            
__________________________________________________________________________________________________
lstm_18 (LSTM)                  (None, 26, 400)      846400      embedding_37[0][0]    

In [ ]:
print ("train")

train


In [ ]:
codesL, commentsL, raw_commentL = data_gen.getTestData()


In [ ]:
codes, partial_captions, next_words, predict_words= data_gen.MakeDataset4(True, mem_size)

In [ ]:
print(codes)
print(partial_captions )
print(predict_words )

[]
[]
[]


In [ ]:
codes = sequence.pad_sequences(codes, maxlen)
partial_captions = sequence.pad_sequences(partial_captions, max_caption_len)
predict_words = sequence.pad_sequences(predict_words, mem_size)

In [ ]:
print(codes)
print(partial_captions )
print(predict_words )

[]
[]
[]


In [ ]:
ids = data_gen.getTestID()

In [ ]:
#def model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, 
 #             callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, 
  #class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, 
  #validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
#del data_gen

epoch = 200
for i in range(epoch):
    model.fit ([codes, partial_captions,predict_words], next_words, batch_size=32, epochs=500,validation_data=([codesT, partial_captionsT,predict_wordsT], next_wordsT),callbacks=[early] )
    json_string = model.to_json()
    open('/content/drive/My Drive/CodeSum/dataset/attmem_qnaData_all.json', 'w').write(json_string)
    model.save_weights('/content/drive/My Drive/CodeSum/dataset/attmem_qnaData_all.h5')
     

IndexError: ignored

In [ ]:
json_string = model.to_json()
open('../model/attmem_qnaData_all.json', 'w').write(json_string)
model.save_weights('../model/attmem_qnaData_all.h5')

FileNotFoundError: ignored

In [ ]:
#########not here!!!
model.load_weights('/content/drive/My Drive/CodeSum/dataset/attmem_qnaData_all.h5')

In [ ]:
    ids = data_gen.getTestID()
    reference_file = "ref_UNK.txt"
    predictions = []
    for j, id in enumerate(ids):
        predictions.append(str(id) + "\t" + gen.printSentence(sens[j][0][1:-1]) + '\n')

IndexError: ignored

Training CodeNN

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import tensorflow.compat.v1 as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True, log_device_placement=True))


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.convolutional import Convolution1D
from tensorflow.python.keras.layers.core import Activation, Flatten, Dense, Dropout, RepeatVector
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense

#import data_generator
from tensorflow.python.keras.preprocessing import sequence
#import keras

from tensorflow.python.keras.models import Model

#from bleu import computeMaps, bleuFromMaps
#from text_generator import SentenceGeneration

In [ ]:
max_caption_len = 26
maxlen = 500
mem_size = 30


In [ ]:
data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_indexed.txt", 0.20,
                                        maxlen, max_caption_len)

codes, partial_captions, next_words = data_gen.MakeDataset3(train=True)


num of data: 4885


In [ ]:
codes = sequence.pad_sequences(codes, maxlen=maxlen)
partial_captions = sequence.pad_sequences(partial_captions, maxlen=max_caption_len)

codesT, partial_captionsT, next_wordsT = data_gen.MakeDataset3(train=False)
codesT = sequence.pad_sequences(codesT, maxlen=maxlen)
partial_captionsT = sequence.pad_sequences(partial_captionsT, maxlen=max_caption_len)


In [ ]:
vocab_size = 5000

max_features = 10000  # size of token

nb_output = data_gen.vocab_size  # len(Y_train[0])

nb_filter = 128
filter_length = 5
hidden_dims = 512
embedding_dims = 512

mem_size = 30

In [ ]:
embedding_for_nse = Embedding(vocab_size,embedding_dims)
###add dropout embedding_for_nse = Embedding(vocab_size,embedding_dims,dropout=0.3)
language_input = Input(shape=(max_caption_len,))
language_embedding = embedding_for_nse(language_input)

In [ ]:
# the output of both models will be tensors of shape (samples, max_caption_len, 128).
# let's encode this vector sequence into a single vector
h = LSTM(hidden_dims, return_sequences=True)(language_embedding)
# which will be used to compute a probability
# distribution over what the next word in the caption should be!

In [ ]:
code_input = Input(shape=(maxlen,))
embedding_for_code = Embedding(max_features, hidden_dims)(code_input)

###add dropout embedding_for_nse = Embedding(vocab_size,embedding_dims,dropout=0.5)
code_match = keras.layers.Dot(axes=[2,2]) (inputs=[h,embedding_for_code])
code_match = Activation('softmax')(code_match)
code_attention = keras.layers.Dot(axes=[2,1])(inputs=[code_match,embedding_for_code])
code_attention = Activation('relu')(code_attention) # mem : memory embedding

In [ ]:
t = Dense(hidden_dims)(code_attention)

h = Dense(hidden_dims)(h)

x = keras.layers.Concatenate(axis=-1) (inputs=[t,h])
x = LSTM(hidden_dims, return_sequences=False)(x)

x = Dense(hidden_dims)(x)

x = Dropout(0.5)(x)
predictions = Dense(nb_output,activation='softmax')(x)
optimizer = RMSprop(lr=0.0001)
model = Model([code_input,language_input], predictions)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 26, 512)      2560000     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 26, 512)      2099200     embedding_3[0][0]                
_______________________________________________________________________________________

In [ ]:
print ("train")

train


In [ ]:
codesL, commentsL, raw_commentL = data_gen.getTestData()
ids = data_gen.getTestID()

del data_gen


max_score = 0

epoch = 200


In [ ]:
#model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, 
 # validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, 
#initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, 
#validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
for i in range(epoch):
    model.fit ([codesT, partial_captionsT], next_wordsT, batch_size=128, epochs=1, validation_split=0.5,validation_data=([codesT, partial_captionsT], next_wordsT),steps_per_epoch=2000,validation_steps=1)
    json_string = model.to_json()
    open('/content/drive/My Drive/CodeSum/dataset/codenn_512.json', 'w').write(json_string)
    model.save_weights('/content/drive/My Drive/CodeSum/dataset/codenn_512.h5')


    reference_file = "ref_UNK.txt"
    predictions = []
    for j, id in enumerate(ids):
        predictions.append(str(id) + "\t" + gen.printSentence(sens[j][0][1:-1]) + '\n')
    del gen
    (goldMap, predictionMap) = computeMaps(predictions, reference_file)
    score = bleuFromMaps(goldMap, predictionMap)[0]

    print ('eter'+str(i))
    print (score)

    if max_score < score and i>5:
        max_score = score
        json_string = model.to_json()
        open('/content/drive/My Drive/CodeSum/dataset/codenn_512.json', 'w').write(json_string)
        model.save_weights('/content/drive/My Drive/CodeSum/dataset/codenn_512.h5')

    print (max_score )

 258/2000 [==>...........................] - 1227s 5s/step - loss: 4.7433 - accuracy: 0.2520 - val_loss: 4.3372 - val_accuracy: 0.2656


KeyError: ignored

In [ ]:
   # del gen


BLEU Score

In [ ]:
#!/usr/bin/python

'''
This script was adapted from the original version by hieuhoang1972 which is part of MOSES.
'''

# $Id: bleu.py 1307 2007-03-14 22:22:36Z hieuhoang1972 $

'''Provides:
cook_refs(refs, n=4): Transform a list of reference sentences as strings into a form usable by cook_test().
cook_test(test, refs, n=4): Transform a test sentence as a string (together with the cooked reference sentences) into a form usable by score_cooked().
score_cooked(alltest, n=4): Score a list of cooked test sentences.
score_set(s, testid, refids, n=4): Interface with dataset.py; calculate BLEU score of testid against refids.
The reason for breaking the BLEU computation into three phases cook_refs(), cook_test(), and score_cooked() is to allow the caller to calculate BLEU scores for multiple test sets as efficiently as possible.
'''

'Provides:\ncook_refs(refs, n=4): Transform a list of reference sentences as strings into a form usable by cook_test().\ncook_test(test, refs, n=4): Transform a test sentence as a string (together with the cooked reference sentences) into a form usable by score_cooked().\nscore_cooked(alltest, n=4): Score a list of cooked test sentences.\nscore_set(s, testid, refids, n=4): Interface with dataset.py; calculate BLEU score of testid against refids.\nThe reason for breaking the BLEU computation into three phases cook_refs(), cook_test(), and score_cooked() is to allow the caller to calculate BLEU scores for multiple test sets as efficiently as possible.\n'

In [ ]:
import sys, math, re, xml.sax.saxutils
import subprocess
import os


In [ ]:
# Added to bypass NIST-style pre-processing of hyp and ref files -- wade
nonorm = 0

preserve_case = False
eff_ref_len = "shortest"

normalize1 = [
    ('<skipped>', ''),  # strip "skipped" tags
    (r'-\n', ''),  # strip end-of-line hyphenation and join lines
    (r'\n', ' '),  # join lines
    #    (r'(\d)\s+(?=\d)', r'\1'), # join digits
]
normalize1 = [(re.compile(pattern), replace) for (pattern, replace) in normalize1]

normalize2 = [
    (r'([\{-\~\[-\` -\&\(-\+\:-\@\/])', r' \1 '),  # tokenize punctuation. apostrophe is missing
    (r'([^0-9])([\.,])', r'\1 \2 '),  # tokenize period and comma unless preceded by a digit
    (r'([\.,])([^0-9])', r' \1 \2'),  # tokenize period and comma unless followed by a digit
    (r'([0-9])(-)', r'\1 \2 ')  # tokenize dash when preceded by a digit
]
normalize2 = [(re.compile(pattern), replace) for (pattern, replace) in normalize2]


In [ ]:
def normalize(s):
    '''Normalize and tokenize text. This is lifted from NIST mteval-v11a.pl.'''
    # Added to bypass NIST-style pre-processing of hyp and ref files -- wade
    if (nonorm):
        return s.split()
    if type(s) is not str:
        s = " ".join(s)
    # language-independent part:
    for (pattern, replace) in normalize1:
        s = re.sub(pattern, replace, s)
    s = xml.sax.saxutils.unescape(s, {'&quot;': '"'})
    # language-dependent part (assuming Western languages):
    s = " %s " % s
    if not preserve_case:
        s = s.lower()  # this might not be identical to the original
    for (pattern, replace) in normalize2:
        s = re.sub(pattern, replace, s)
    return s.split()


In [ ]:
def count_ngrams(words, n=4):
    counts = {}
    for k in xrange(1, n + 1):
        for i in xrange(len(words) - k + 1):
            ngram = tuple(words[i:i + k])
            counts[ngram] = counts.get(ngram, 0) + 1
    return counts

In [ ]:
def cook_refs(refs, n=4):
    '''Takes a list of reference sentences for a single segment
    and returns an object that encapsulates everything that BLEU
    needs to know about them.'''

    refs = [normalize(ref) for ref in refs]
    maxcounts = {}
    for ref in refs:
        counts = count_ngrams(ref, n)
        for (ngram, count) in counts.iteritems():
            maxcounts[ngram] = max(maxcounts.get(ngram, 0), count)
    return ([len(ref) for ref in refs], maxcounts)

In [ ]:
def cook_test(test,reflens, refmaxcounts, n=4):
    '''Takes a test sentence and returns an object that
    encapsulates everything that BLEU needs to know about it.'''

    test = normalize(test)
    result = {}
    result["testlen"] = len(test)

    # Calculate effective reference sentence length.

    if eff_ref_len == "shortest":
        result["reflen"] = min(reflens)
    elif eff_ref_len == "average":
        result["reflen"] = float(sum(reflens)) / len(reflens)
    elif eff_ref_len == "closest":
        min_diff = None
        for reflen in reflens:
            if min_diff is None or abs(reflen - len(test)) < min_diff:
                min_diff = abs(reflen - len(test))
                result['reflen'] = reflen

    result["guess"] = [max(len(test) - k + 1, 0) for k in xrange(1, n + 1)]

    result['correct'] = [0] * n
    counts = count_ngrams(test, n)
    for (ngram, count) in counts.iteritems():
        result["correct"][len(ngram) - 1] += min(refmaxcounts.get(ngram, 0), count)

    return result

In [ ]:
def score_cooked(allcomps, n=4, ground=0, smooth=1):
    totalcomps = {'testlen': 0, 'reflen': 0, 'guess': [0] * n, 'correct': [0] * n}
    for comps in allcomps:
        for key in ['testlen', 'reflen']:
            totalcomps[key] += comps[key]
        for key in ['guess', 'correct']:
            for k in xrange(n):
                totalcomps[key][k] += comps[key][k]
    logbleu = 0.0
    all_bleus = []
    for k in xrange(n):
        correct = totalcomps['correct'][k]
        guess = totalcomps['guess'][k]
        addsmooth = 0
        if smooth == 1 and k > 0:
            addsmooth = 1
        logbleu += math.log(correct + addsmooth + sys.float_info.min) - math.log(guess + addsmooth)
        if guess == 0:
            all_bleus.append(-10000000)
        else:
            all_bleus.append(math.log(correct + sys.float_info.min) - math.log(guess))

    logbleu /= float(n)
    all_bleus.insert(0, logbleu)

    brevPenalty = min(0, 1 - float(totalcomps['reflen'] + 1) / (totalcomps['testlen'] + 1))
    for i in xrange(len(all_bleus)):
        if i == 0:
            all_bleus[i] += brevPenalty
        all_bleus[i] = math.exp(all_bleus[i])
    return all_bleus

In [ ]:
def bleu(refs, candidate, ground=0, smooth=1):
    refs = cook_refs(refs)
    test = cook_test(candidate, refs)
    return score_cooked([test], ground=ground, smooth=smooth)



In [ ]:
def splitPuncts(line):
    return ' '.join(re.findall(r"[\w]+|[^\s\w]", line))

In [ ]:
def computeMaps(predictions, goldfile):
    predictionMap = {}
    goldMap = {}
    gf = open(goldfile, 'r')

    for row in predictions:
        cols = row.strip().split('\t')
        if len(cols) == 1:
            (rid, pred) = (cols[0], '')
        else:
            (rid, pred) = (cols[0], cols[1])
        predictionMap[rid] = [splitPuncts(pred.strip().lower())]

    for row in gf:
        (rid, pred) = row.split('\t')
        if rid in predictionMap:  # Only insert if the id exists for the method
            if rid not in goldMap:
                goldMap[rid] = []
            goldMap[rid].append(splitPuncts(pred.strip().lower()))

    sys.stderr.write('Total: ' + str(len(goldMap)) + '\n')
    return (goldMap, predictionMap)

In [ ]:
# m1 is the reference map
# m2 is the prediction map
def bleuFromMaps(m1, m2):
    score = [0] * 5
    num = 0.0

    for key in m1:
        if key in m2:
            bl = bleu(m1[key], m2[key][0])
            score = [score[i] + bl[i] for i in range(0, len(bl))]
            num += 1
    return [s * 100.0 / num for s in score]


In [ ]:
if __name__ == '__main__':

    file = 'predict_1_codenn_512'

    ids = []
    with open('/content/drive/My Drive/CodeSum/codenn/data/stackoverflow/csharp/dev/codenn.txt') as f:
        ids = [val.split('\t')[0] for val in f]

    reference_file = "/content/drive/My Drive/CodeSum/generator/ref.txt"
    predictions = []
    for i,row in enumerate(open(file+'.txt')):
        r = str(ids[i])+"\t"+row.split('\t')[1]
        predictions.append(r)
    (goldMap, predictionMap) = computeMaps(predictions, reference_file)
    print (bleuFromMaps(goldMap, predictionMap)[0])
    print (bleuFromMaps(goldMap, predictionMap)[1])
    print (bleuFromMaps(goldMap, predictionMap)[2])
    print (bleuFromMaps(goldMap, predictionMap)[3])
    print (bleuFromMaps(goldMap, predictionMap)[4])

    reference_file = "ref_UNK.txt"
    predictions = []
    for i, row in enumerate(open(file+'.txt')):
        r = str(ids[i]) + "\t" + row.split('\t')[1]
        predictions.append(r)
    (goldMap, predictionMap) = computeMaps(predictions, reference_file)
    print (bleuFromMaps(goldMap, predictionMap)[0])
    print (bleuFromMaps(goldMap, predictionMap)[1])
    print (bleuFromMaps(goldMap, predictionMap)[2])
    print (bleuFromMaps(goldMap, predictionMap)[3])
    print (bleuFromMaps(goldMap, predictionMap)[4])

Total: 0


ZeroDivisionError: ignored

Text Generation

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import tensorflow.compat.v1 as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True, log_device_placement=True))


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



In [ ]:
from tensorflow.python.keras.models import model_from_json, Sequential
import numpy as np
import math
import logging
import sys
import nltk
from tensorflow.python.keras.models import Sequential
#import data_generator
from tensorflow.python.keras.preprocessing import sequence


In [ ]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [ ]:
class SentenceGeneration(object):
    def __init__(self):
       # pp=CGModel(20000, 600, 100, 200, 4000 , 'rmsprop', None,False, -1, None,None, 0.001)
        self.model =  Sequential()              #pp.buildModel() 
        self.index2word = dict()
        self.word2Index = dict()
        self.index2token = dict()
        self.token2Index = dict()

        self.predict_model = self.readKeywordModel('keyword_f')
        ### in place keyword_f q_keyword2

        with open("/content/drive/My Drive/CodeSum/dataset/comment_f_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                word = word.rstrip()
                self.index2word[i] = word
                self.word2Index[word] = int(i)
        with open("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_Vocab.txt") as fin:
            for i, word in enumerate(fin):
                word = word.rstrip()
                self.index2token[i] = word
                self.token2Index[word] = int(i)

    # def __init__(self,codes,targets):
    #     self.model = Sequential()
    #     # self.codes = codes
    #     # self.targets = targets

    def readModel(self, name):
        model = model_from_json(open('/content/drive/My Drive/CodeSum/dataset/'+name + '.json').read())
        model.load_weights('/content/drive/My Drive/CodeSum/dataset/'+name + '.h5')
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])
        self.model = model

    def readKeywordModel(self, name):
        model = model_from_json(open('/content/drive/My Drive/CodeSum/dataset/' + name + '.json').read())
        model.load_weights('/content/drive/My Drive/CodeSum/dataset/' + name + '.h5')
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])
        return model

    def setModel(self, model):
        self.model = model

    def printSentence(self, indices):
        return ' '.join([self.index2word[x] for x in indices])

        #for index in indices:
        #    print self.index2word[index],
    def returnCode(self, indices):

        return ' '.join([self.index2token[x-1] for x in indices])

    def removeToken(self,indices):
        indices = indices[1:-1]
        unk_index = self.word2Index['UNK']
        if unk_index in indices:
            indices.remove(unk_index)
        return indices





    def generateSentence(self, code,beam_size,nonUNK,useMemory):

        code = sequence.pad_sequences([code], maxlen=500)

        if useMemory:
            keyword_pred = self.predict_model.model.predict(code)[0]

            keyword_sorted_pred = np.argsort(keyword_pred)[::-1]
            keywords = [s for s in keyword_sorted_pred[:30]]
            keywords = sequence.pad_sequences([keywords], maxlen=30)


        start_index = self.word2Index['START']
        end_index = self.word2Index['END']
        caption = [[start_index]]
        caption = sequence.pad_sequences(caption, maxlen=26)

        if useMemory:
            pred = self.model.predict([code, caption, keywords])[0]
        else:
            pred = self.model.predict([code, caption])[0]
        sorted_pred = np.argsort(pred)[::-1]

        # initial
        beams = []
        for index in sorted_pred[:beam_size]:
            beams.append((math.log(pred[index]), [start_index, index]))

        finished = []

        for t in range(1, 26):  # max_caption_len
            # generate
            candidates = []
            for b in beams:
                caption = sequence.pad_sequences([b[1]], maxlen=26)
                if useMemory:
                    pred = self.model.predict([code, caption, keywords])[0]
                else:
                    pred = self.model.predict([code, caption])[0]
                sorted_pred = np.argsort(pred)[::-1]

                if nonUNK:
                    candi_index = [val for val in sorted_pred[:beam_size]]
                    if self.word2Index['UNK'] in candi_index:
                        candi_index.append(sorted_pred[beam_size+1])
                else:
                    candi_index = sorted_pred[:beam_size]


                for index in candi_index:
                    if nonUNK:
                        if self.word2Index['UNK'] == index:
                            continue

                    new_caption = b[1][:]
                    new_caption.append(index)
                    candidates.append([b[0] + math.log(pred[index]), new_caption])
            candidates.sort(reverse=True)
            beams = candidates[:beam_size]

            # end sentence
            unfinished = []
            for b in beams:
                if b[1][-1] == end_index:
                    finished.append(b)
                    beam_size -= 1
                else:
                    unfinished.append(b)
            beams = unfinished[:]

            # print ""
            # for b in beams:
            #     print "%5f" % b[0], ' '.join([self.index2word[key] for key in b[1]])

            if beam_size == 0:
                break

        for i in range(beam_size):
            finished.append(beams[i])

        # re-ranking
        for f in finished:
            f[0] = f[0] / len(f[1])
            finished.sort(reverse=True)

        # print "---generate sentences---"
        s = []
        for b in finished:
            #sen = ' '.join([self.index2word[key] for key in b[1]])
            s.append(b[1])
        return s

In [ ]:
from textgenrnn import textgenrnn

In [ ]:
 import tensorflow as tf

In [ ]:
!pip install textgenrnn

In [ ]:
if __name__ == '__main__':
    gen = SentenceGeneration()
    gen.readModel('codenn_512')
    #### replace keyword_f with codenn_512
    name = 'codenn_512'
    nonUNK = 1
    beamSize = 10
    useMemory = 1

    data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_indexed.txt",
                                            0.20, 500, 26)

    codes,comments,raw_comment= data_gen.getTestData()
    '''
    np.random.seed(30)
    np.random.shuffle(codes)
    np.random.seed(30)
    np.random.shuffle(raw_comment)
    np.random.seed(30)
    np.random.shuffle(comments)
    '''
    sum_of_BLEU1 = 0
    sum_of_BLEU2 = 0
    sum_of_BLEU3 = 0
    sum_of_BLEU4 = 0
    j = 0

    sens = []
    co = []
    comm = []
    raw_com=[]

    score = []

    for code,comment,raw in zip(codes,comments,raw_comment):
        print(code)
        sentences = gen.generateSentence(code,beamSize,nonUNK,0)



        sens.append(sentences)
        co.append(gen.returnCode(code))
        comm.append(gen.printSentence(comment[1:-1]))
        raw_com.append(raw)
        #print "origen"
        #print gen.printSentence(comment)
        #print "generate"
        #for s in sentences:
        #    print gen.printSentence(s)
        #print comment
        #print sentences[0]
        # s = gen.removeToken(sentences[0])
        # c = gen.removeToken(comment)
        s = sentences[0][1:-1]
        c = comment[1:-1]

        BLEU1_score = nltk.translate.bleu_score.modified_precision([c], s, n=1)
        BLEU2_score = nltk.translate.bleu_score.modified_precision([c], s, n=2)
        BLEU3_score = nltk.translate.bleu_score.modified_precision([c], s, n=3)
        BLEU4_score = nltk.translate.bleu_score.modified_precision([c], s, n=4)

        sum_of_BLEU1 += float(BLEU1_score)
        sum_of_BLEU2 += float(BLEU2_score)
        sum_of_BLEU3 += float(BLEU3_score)
        sum_of_BLEU4 += float(BLEU4_score)
        score.append([BLEU1_score,j])
        j += 1


        sys.stdout.write('\r' + str(j)+' score :'+str(sum_of_BLEU1 / float(j)))


        #print 'score :', sum_of_BLEU / float(i)
        #print i,float(BLEU_score)


num of data: 4885
[1, 2, 9, 10, 11, 12, 6, 13, 11, 12, 14, 13, 15, 7, 8]


KeyboardInterrupt: ignored

In [ ]:
for code,comment,raw in zip(codes,comments,raw_comment)[:]:
        sentences = gen.generateSentence(code,beamSize,nonUNK=nonUNK,useMemory=0)



        sens.append(sentences)
        co.append(gen.returnCode(code))
        comm.append(gen.printSentence(comment[1:-1]))
        raw_com.append(raw)
        #print "origen"
        #print gen.printSentence(comment)
        #print "generate"
        #for s in sentences:
        #    print gen.printSentence(s)
        #print comment
        #print sentences[0]
        # s = gen.removeToken(sentences[0])
        # c = gen.removeToken(comment)
        s = sentences[0][1:-1]
        c = comment[1:-1]

        BLEU1_score = nltk.translate.bleu_score.modified_precision([c], s, n=1)
        BLEU2_score = nltk.translate.bleu_score.modified_precision([c], s, n=2)
        BLEU3_score = nltk.translate.bleu_score.modified_precision([c], s, n=3)
        BLEU4_score = nltk.translate.bleu_score.modified_precision([c], s, n=4)

        sum_of_BLEU1 += float(BLEU1_score)
        sum_of_BLEU2 += float(BLEU2_score)
        sum_of_BLEU3 += float(BLEU3_score)
        sum_of_BLEU4 += float(BLEU4_score)
        score.append([BLEU1_score,j])
        j += 1


        sys.stdout.write('\r' + str(j)+' score :'+str(sum_of_BLEU1 / float(j)))


        #print 'score :', sum_of_BLEU / float(i)
        #print i,float(BLEU_score)




    score.sort(reverse=True)
    #high_score_index = [index[1] for index in score]

    with open('code_rand_'+name+'.txt', 'w') as fin:
        fin.write(str(sum_of_BLEU1 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU2 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU3 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU4 / float(j)))
        fin.write("\n")
        for i in range(len(co[:])):
            s = str(co[i])
            code = s.replace("{","{\n").replace(";",";\n").replace("}","}\n")
            fin.write("code:\n"+ code)
            fin.write("comment:\n"+ comm[i].rstrip()+ '\n')
            fin.write("--generate--\n")
            for s in sens[i]:
                fin.write(gen.printSentence(s[1:-1])+'\n')
            fin.write("\n")

    with open('code_sort_'+name+'.txt', 'w') as fin:
        fin.write(str(sum_of_BLEU1 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU2 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU3 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU4 / float(j)))
        fin.write("\n")

        for (sc,i) in score:
            s = str(co[i])
            code = s.replace("{", "{\n").replace(";", ";\n").replace("}", "}\n")
            fin.write("code:\n" + code)
            fin.write("comment:\n" + comm[i].rstrip() + '\n')
            fin.write("BLUE1:"+ str(float(sc))+ '\n')
            fin.write("--generate--\n")
            for s in sens[i]:
                fin.write(gen.printSentence(s[1:-1]) + '\n')
            fin.write("\n")


    with open('gold.txt', 'w') as fin:
        for i in range(len(co[:])):
            fin.write(str(i)+"\t"+comm[i].rstrip()+ '\n')
    with open('predict_n_'+name+'.txt', 'w') as fin:
        for i in range(len(co[:])):
            for s in sens[i]:
                fin.write(str(i)+"\t"+gen.printSentence(s[1:-1])+'\n')


    with open('predict_1_'+name+'.txt', 'w') as fin:
        for i in range(len(co[:])):
            fin.write(str(i)+"\t"+gen.printSentence(sens[i][0][1:-1])+'\n')

In [ ]:
if __name__ == '__main__':
    gen = SentenceGeneration()
    gen.readModel('codenn_512')
    name = 'codenn_512'
    nonUNK = 1
    beamSize = 10
    useMemory = 1

    data_gen = DataGenerator("/content/drive/My Drive/CodeSum/dataset/code_f_keyword_indexed.txt", "/content/drive/My Drive/CodeSum/dataset/comment_f_indexed.txt",
                                            0.20, 500, 26)

    codes,comments,raw_comment= data_gen.getTestData()
    '''
    np.random.seed(30)
    np.random.shuffle(codes)
    np.random.seed(30)
    np.random.shuffle(raw_comment)
    np.random.seed(30)
    np.random.shuffle(comments)
    '''
    sum_of_BLEU1 = 0
    sum_of_BLEU2 = 0
    sum_of_BLEU3 = 0
    sum_of_BLEU4 = 0
    j = 0

    sens = []
    co = []
    comm = []
    raw_com=[]

    score = []

    for code,comment,raw in zip(codes,comments,raw_comment):
        sentences = gen.generateSentence(code,beamSize,nonUNK=nonUNK,useMemory=0)



        sens.append(sentences)
        co.append(gen.returnCode(code))
        comm.append(gen.printSentence(comment[1:-1]))
        raw_com.append(raw)
        #print "origen"
        #print gen.printSentence(comment)
        #print "generate"
        #for s in sentences:
        #    print gen.printSentence(s)
        #print comment
        #print sentences[0]
        # s = gen.removeToken(sentences[0])
        # c = gen.removeToken(comment)
        s = sentences[0][1:-1]
        c = comment[1:-1]

        BLEU1_score = nltk.translate.bleu_score.modified_precision([c], s, n=1)
        BLEU2_score = nltk.translate.bleu_score.modified_precision([c], s, n=2)
        BLEU3_score = nltk.translate.bleu_score.modified_precision([c], s, n=3)
        BLEU4_score = nltk.translate.bleu_score.modified_precision([c], s, n=4)

        sum_of_BLEU1 += float(BLEU1_score)
        sum_of_BLEU2 += float(BLEU2_score)
        sum_of_BLEU3 += float(BLEU3_score)
        sum_of_BLEU4 += float(BLEU4_score)
        score.append([BLEU1_score,j])
        j += 1


        sys.stdout.write('\r' + str(j)+' score :'+str(sum_of_BLEU1 / float(j)))


        #print 'score :', sum_of_BLEU / float(i)
        #print i,float(BLEU_score)




    score.sort(reverse=True)
    #high_score_index = [index[1] for index in score]

    with open('/content/drive/My Drive/CodeSum/dataset/code_rand_'+name+'.txt', 'w') as fin:
        fin.write(str(sum_of_BLEU1 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU2 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU3 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU4 / float(j)))
        fin.write("\n")
        for i in range(len(co[:])):
            s = str(co[i])
            code = s.replace("{","{\n").replace(";",";\n").replace("}","}\n")
            fin.write("code:\n"+ code)
            fin.write("comment:\n"+ comm[i].rstrip()+ '\n')
            fin.write("--generate--\n")
            for s in sens[i]:
                fin.write(gen.printSentence(s[1:-1])+'\n')
            fin.write("\n")

    with open('/content/drive/My Drive/CodeSum/dataset/code_sort_'+name+'.txt', 'w') as fin:
        fin.write(str(sum_of_BLEU1 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU2 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU3 / float(j)))
        fin.write("\n")
        fin.write(str(sum_of_BLEU4 / float(j)))
        fin.write("\n")

        for (sc,i) in score:
            s = str(co[i])
            code = s.replace("{", "{\n").replace(";", ";\n").replace("}", "}\n")
            fin.write("code:\n" + code)
            fin.write("comment:\n" + comm[i].rstrip() + '\n')
            fin.write("BLUE1:"+ str(float(sc))+ '\n')
            fin.write("--generate--\n")
            for s in sens[i]:
                fin.write(gen.printSentence(s[1:-1]) + '\n')
            fin.write("\n")


    with open('/content/drive/My Drive/CodeSum/dataset/gold.txt', 'w') as fin:
        for i in range(len(co[:])):
            fin.write(str(i)+"\t"+comm[i].rstrip()+ '\n')
    with open('/content/drive/My Drive/CodeSum/dataset/predict_n_'+name+'.txt', 'w') as fin:
        for i in range(len(co[:])):
            for s in sens[i]:
                fin.write(str(i)+"\t"+gen.printSentence(s[1:-1])+'\n')


    with open('/content/drive/My Drive/CodeSum/dataset/predict_1_'+name+'.txt', 'w') as fin:
        for i in range(len(co[:])):
            fin.write(str(i)+"\t"+gen.printSentence(sens[i][0][1:-1])+'\n')

num of data: 4885
489 score :0.3100715746421268